In [ ]:
import os
from src.StreamPort.core import Engine
from src.StreamPort.device.analyses import PressureCurvesAnalyses

path = "C:/Users/Sandeep/Desktop/ExtractedSignals" 
batches = os.listdir(path)
batches = [os.path.join(path, file) for file in batches]

files = []
for batch in batches:
    batch_files = os.listdir(batch)
    batch_files = [os.path.join(batch, file) for file in batch_files if ".D" in file]
    files.extend(batch_files)
print(files)

In [ ]:

eng = Engine(analyses = PressureCurvesAnalyses(files=files))

print("Number of analyses: ", len(eng.analyses.data))

fig_methods = eng.analyses.plot_methods()

fig_methods.write_image("dev/figures/fig_methods.png", width=1100, height= 350, scale = 3)

fig_methods

In [ ]:
#store variables in the environment to be reused across notebooks (see dev_workflow). If %store is executed once, variable remains valid even after kernel reset, needs only to be retrieved.
#%store files
#pc = eng.analyses
#%store pc
#%store -z

In [ ]:
from src.StreamPort.device.methods import PressureCurvesMethodExtractFeaturesNative

eng.workflow.clear()
eng.workflow.append(PressureCurvesMethodExtractFeaturesNative(period=10, bins=6, window_size=7)) # defaults: (period = 10, window_size = 7, bins = 4, crop = 2)
eng.run()

method = "SAA_411_Pac.M"
method_indices = eng.analyses.get_method_indices(method)
fig_sel_method = eng.analyses.plot_methods(method_indices)

fig_sel_method.write_image("dev/figures/fig_sel_method.png", width=1100, height= 350, scale = 3)
fig_sel_method.write_image("dev/figures/fig_sel_method_half.png", width=550, height= 350, scale = 3)

fig_sel_method.show()

In [ ]:
fig_curves_raw = eng.analyses.plot_pressure_curves(indices = method_indices)

fig_curves_raw.update_layout(showlegend=True)

# for trace in fig_curves_raw.data:
#     trace.line.color = "black"

fig_curves_raw.write_image("dev/figures/fig_curves_raw.png", width=1100, height= 350, scale = 3)

fig_curves_raw.show()

In [ ]:
print("Number of curves: ", len(method_indices))

fig_features=eng.analyses.plot_features(indices = method_indices)

fig_features.update_layout(showlegend=True)
# for trace in fig_features.data:
#     trace.line.color = "black"

fig_features.write_image("dev/figures/fig_features.png", width=1100, height= 350, scale = 3)

fig_features.show()

In [ ]:
import datetime
date_threshold = "2021-08-18"
date_threshold = datetime.datetime.strptime(date_threshold, "%Y-%m-%d")

train_indices = []
for i in method_indices:
    meta = eng.analyses.get_metadata(i)
    batch_position = meta["batch_position"].item()
    start_time = meta["start_time"].item()
    if isinstance(start_time, str):
        start_time = datetime.datetime.fromisoformat(start_time)
    if batch_position > 5 and start_time < date_threshold:
        train_indices.append(i)

train_data = eng.analyses.get_features(train_indices)
train_metadata = eng.analyses.get_metadata(train_indices)
train_data.to_csv("dev/train_features.csv", index=False)
train_metadata.to_csv("dev/train_metadata.csv", index=False)
print("Number of training curves: ", len(train_indices))

In [ ]:
fig_train=eng.analyses.plot_pressure_curves(indices = train_indices)
fig_train.update_layout(showlegend=True)
for trace in fig_train.data:
    trace.line.color = "black"
fig_train.write_image("dev/figures/fig_train.png", width=1100, height= 350, scale = 3)
fig_train.show()

In [ ]:
fig_train_features=eng.analyses.plot_features(indices = train_indices)
fig_train_features.update_layout(showlegend=False)
for trace in fig_train_features.data:
    trace.line.color = "black"
fig_train_features.write_image("dev/figures/fig_train_features.png", width=1100, height= 350, scale = 3)
fig_train_features.show()
print("Train set: ", train_indices)

In [ ]:
tests = {}

In [ ]:
import datetime
date_threshold_min = "2021-08-18"
date_threshold_min= datetime.datetime.strptime(date_threshold_min, "%Y-%m-%d")

date_threshold_max = "2021-08-22"
date_threshold_max= datetime.datetime.strptime(date_threshold_max, "%Y-%m-%d")

test_indices = []
for i in method_indices:
    meta = eng.analyses.get_metadata(i)
    batch_position = meta["batch_position"].item()
    start_time = meta["start_time"].item()
    if isinstance(start_time, str):
        start_time = datetime.datetime.fromisoformat(start_time)
    if start_time < date_threshold_max and start_time > date_threshold_min and batch_position > 0:
        test_indices.append(i)

#tests[f"{date_threshold_min} - {date_threshold_max}"] = test_indices
#%store tests
#test_indices.remove(116) #remove extreme anomalies and test sensitivity of the model
test_data = eng.analyses.get_features(test_indices)
test_metadata = eng.analyses.get_metadata(test_indices)
test_data.to_csv("dev/test_features.csv", index=False)
test_metadata.to_csv("dev/test_metadata.csv", index=False)
print("Number of test curves: ", len(test_indices))

fig_test=eng.analyses.plot_pressure_curves(indices = test_indices)
fig_test.update_layout(showlegend=False)
#for trace in fig_outliers.data:
#    trace.line.color = "black"
fig_test.write_image("dev/figures/fig_test.png", width=1100, height= 350, scale = 3)
fig_test.update_layout(showlegend=True)
fig_test.show()

In [ ]:
fig_test = eng.analyses.plot_features(indices = test_indices, normalize=False)
fig_test.update_layout(showlegend=False)
# for trace in fig_text.data:
#     trace.line.color = "black"
fig_test.write_image("dev/figures/fig_test_features.png", width=1100, height= 350, scale = 3)
fig_test.update_layout(showlegend=True)
fig_test.show()

In [ ]:
fig_test_features_raw = eng.analyses.plot_features_raw(indices = test_indices)
fig_test_features_raw.update_layout(showlegend=False)
fig_test_features_raw.write_image("dev/figures/fig_test_features_raw.png", width=1100, height= 350, scale = 3)
fig_test_features_raw.update_layout(showlegend=True)
fig_test_features_raw.show()

In [ ]:
import datetime
date_threshold_min = "2021-09-09"
date_threshold_min= datetime.datetime.strptime(date_threshold_min, "%Y-%m-%d")

date_threshold_max = "2021-09-12"
date_threshold_max= datetime.datetime.strptime(date_threshold_max, "%Y-%m-%d")

test2_indices = []
for i in method_indices:
    meta = eng.analyses.get_metadata(i)
    batch_position = meta["batch_position"].item()
    start_time = meta["start_time"].item()
    if isinstance(start_time, str):
        start_time = datetime.datetime.fromisoformat(start_time)
    if start_time < date_threshold_max and start_time > date_threshold_min and batch_position > 0:	
        test2_indices.append(i)

# test2_indices = test2_indices[:1]
# test2_indices.append(47)
# test2_indices.append(17)
# test2_indices.append(119)
# test2_indices.append(16)
#pc = eng.analyses
#%store pc tests

test2_data = eng.analyses.get_features(test2_indices)
test2_metadata = eng.analyses.get_metadata(test2_indices)
test2_data.to_csv("dev/test2_features.csv", index=False)
test2_metadata.to_csv("dev/test2_metadata.csv", index=False)
print("Number of test curves: ", len(test2_indices))

fig_test2=eng.analyses.plot_pressure_curves(indices = test2_indices)
fig_test2.update_layout(showlegend=False)
#for trace in fig_outliers.data:
#    trace.line.color = "black"
fig_test2.write_image("dev/figures/fig_test2.png", width=1100, height= 350, scale = 3)
fig_test2.update_layout(showlegend=True)
fig_test2.show()

In [ ]:
fig_test2 = eng.analyses.plot_features(indices = test2_indices)
fig_test2.update_layout(showlegend=False)
# for trace in fig_text.data:
#     trace.line.color = "black"
fig_test2.write_image("dev/figures/fig_test2_features.png", width=1100, height= 350, scale = 3)
fig_test2.update_layout(showlegend=True)
fig_test2.show()

In [ ]:
fig_test2_features_raw = eng.analyses.plot_features_raw(indices = test2_indices)
fig_test2_features_raw.update_layout(showlegend=False)
fig_test2_features_raw.write_image("dev/figures/fig_test2_features_raw.png", width=1100, height= 350, scale = 3)
fig_test2_features_raw.update_layout(showlegend=True)
fig_test2_features_raw.show()